In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
import requests
from bs4 import BeautifulSoup
import time
import openpyxl
import mysql.connector


# Conectarse a la base de datos
conexion = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="",
    database="ligafantasyy"
)
cursor = conexion.cursor()

def click_mas():
    # Pinchar en el botón del menu "Más"
    masMenu = driver.find_element(By.XPATH, '//*[@id="content"]/header/div[2]/ul/li[5]/a')

    try:
        masMenu.click()
    except (ElementNotInteractableException, NoSuchElementException):
        # Maneja la excepción y espera antes de intentar nuevamente
        print("Anuncio detectado, reiniciando driver...")
        driver.refresh()
        time.sleep(3) 
        masMenu.click()
        
        
# Función para obtener el valor basado en la etiqueta
def obtener_valor_por_etiqueta(label_deseado):
    elemento = driver.find_element(By.XPATH, f"//div[@class='item']//div[@class='label' and text()='{label_deseado}']/following-sibling::div[@class='value']")
    valor = elemento.text
    return valor
teams_data = {
    "Real Madrid": "https://cdn.gomister.com/file/cdn-common/teams/15.png?version=20231117",
    "Real Sociedad": "https://cdn.gomister.com/file/cdn-common/teams/16.png?version=20231117",
    "Atlético de Madrid": "https://cdn.gomister.com/file/cdn-common/teams/2.png?version=20231117",
    "Girona": "https://cdn.gomister.com/file/cdn-common/teams/222.png?version=20231117",
    "Osasuna": "https://cdn.gomister.com/file/cdn-common/teams/50.png?version=20231117",
    "Athletic Club": "https://cdn.gomister.com/file/cdn-common/teams/1.png?version=20231117",
    "Valencia": "https://cdn.gomister.com/file/cdn-common/teams/19.png?version=20231117",
    "Granada": "https://cdn.gomister.com/file/cdn-common/teams/10.png?version=20231117",
    "Getafe": "https://cdn.gomister.com/file/cdn-common/teams/9.png?version=20231117",
    "Villarreal": "https://cdn.gomister.com/file/cdn-common/teams/20.png?version=20231117",
    "Las Palmas": "https://cdn.gomister.com/file/cdn-common/teams/11.png?version=20231117",
    "Mallorca": "https://cdn.gomister.com/file/cdn-common/teams/408.png?version=20231117",
    "Rayo Vallecano": "https://cdn.gomister.com/file/cdn-common/teams/14.png?version=20231117",
    "Barcelona": "https://cdn.gomister.com/file/cdn-common/teams/3.png?version=20231117",
    "Celta de Vigo": "https://cdn.gomister.com/file/cdn-common/teams/5.png?version=20231117",
    "Cádiz": "https://cdn.gomister.com/file/cdn-common/teams/499.png?version=20231117",
    "Alavés": "https://cdn.gomister.com/file/cdn-common/teams/48.png?version=20231117",
    "Almería": "https://cdn.gomister.com/file/cdn-common/teams/21.png?version=20231117",
    "Sevilla": "https://cdn.gomister.com/file/cdn-common/teams/17.png?version=20231117",
    "Betis": "https://cdn.gomister.com/file/cdn-common/teams/4.png?version=20231117",
}



def actualizar_version(teams_data, version):
      for equipo, url in teams_data.items():
        # Dividir la URL en base al signo de interrogación
        partes = url.split('?')
        
        # Verificar si hay una parte después del signo de interrogación y actualizar la versión
        if len(partes) > 1:
            partes[1] = f"version={version}"
            
            # Volver a unir las partes para formar la URL actualizada
            nueva_url = '?'.join(partes)
            
            # Actualizar la URL en el diccionario
            teams_data[equipo] = nueva_url

        #print(version)
        #print("nuevaaaa url-->  ",nueva_url)
        
        
        
        
        
def extraer_info_jugador(jornada_absolute,jornada_a_scrapear):
    
    
    nombre = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[2]")
    apellido = driver.find_element(By.XPATH, " /html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[3]")
    valorS=driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div[2]/div[2]/div/div/div[1]/div[2]')
    valor=valorS.text
        
    media_puntos_local = obtener_valor_por_etiqueta("Media en casa")
    media_puntos_visitante = obtener_valor_por_etiqueta("Media fuera")
    try:
        edad = obtener_valor_por_etiqueta("Edad")
        altura = obtener_valor_por_etiqueta("Altura")
        peso = obtener_valor_por_etiqueta("Peso")
    except:
        edad = None
        altura = None
        peso = None
            
    if peso == "kg":
        peso = None
        
        
    #### OBTENER EQUIPO JUGADOR ####
    
    # Obtener src del equipo
    team_logo_element = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[1]/a/img")
    image_url = team_logo_element.get_attribute("src")

    # Comparar la URL de la imagen con las URLs en teams_data
    equipo = None
    proximo_rival=None
    local= False
    for equipo_nombre, equipo_url in teams_data.items():
        if image_url == equipo_url:
            equipo = equipo_nombre
            
            #### OBTENER RESULTADO ÚLTIMO PARTIDO ####
            try:
                divpartido = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[3]/div[1]/div[3]/div")
            except:
                divpartido = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[3]/div/div[2]/div")
            
            # Encuentra el div del partido
            item_elements = divpartido.find_elements(By.CLASS_NAME, 'item')
           
            # Encuentra las imágenes dentro del div partido
            img_elements = item_elements[0].find_elements(By.CLASS_NAME, 'team-logo')

            # Guarda las src de las imágenes en variables
            if len(img_elements) >= 2:
                src_img1 = img_elements[0].get_attribute('src')
                src_img2 = img_elements[1].get_attribute('src')
                if src_img1 == image_url:
                    local = True
                    for equipo_nombre, equipo_url in teams_data.items():
                        if src_img2 == equipo_url:
                            proximo_rival=equipo_nombre
                else:
                    local=False
                    for equipo_nombre, equipo_url in teams_data.items():
                        if src_img1 == equipo_url:
                            proximo_rival=equipo_nombre
            else:
                print("No se encontro el próximo partido")
            

    #### OBTENER POSICIÓN DEL JUGADOR ####

    elemento = driver.find_element(By.XPATH, '//i[contains(@class, "pos-")]')
    # Obtener el valor del atributo class
    clases = elemento.get_attribute("class").split()

    # Determinar la posición
    posicion = None
    for clase in clases:
        if clase.startswith("pos-") and "pos-big" in clases:
            if clase == "pos-1":
                posicion = "PT"
            elif clase == "pos-2":
                posicion = "DF"
            elif clase == "pos-3":
                posicion = "MC"
            elif clase == "pos-4":
                posicion = "DL"
            break
            
            
    #### OBTENER PUNTOS DEL JUGADOR ####
    
    # Encontrar jornada 
    elementos_principales = driver.find_elements(By.CLASS_NAME, 'btn-player-gw')

    # Iterar sobre cada elemento encontrado
    subelemento_gw=None
    for elemento_principal in elementos_principales:
        # Encontrar subelemento con la clase 'gw' dentro de cada elemento principal
        subelemento_gw = elemento_principal.find_element(By.CLASS_NAME, 'gw')

        # Verificar si el texto coincide con el de la jornada
        if subelemento_gw.text == jornada_a_scrapear:
            jornada_name = subelemento_gw.text
            break             
            
    if jornada_name ==jornada_absolute:
        # Encontrar jornada en la web con otro elemennto como referencia
        localizador = driver.find_element(By.XPATH, "//h4[text()='Valor']")
        driver.execute_script("arguments[0].scrollIntoView(true);", localizador)   

        time.sleep(1)
        try:
            subelemento_gw.click()
        except:
            elemento_principal.click()

        time.sleep(2)
        try:
            # PUNTOS MISTER FANTASY
            main_provider = driver.find_element(By.CLASS_NAME, 'main-provider')
            points_element = main_provider.find_element(By.CLASS_NAME, 'points')
            final_points = points_element.get_attribute('data-points')

            # PUNTOS AS, MARCA Y MUNDO DEPORTIVO 
            providers_div = driver.find_element(By.CLASS_NAME, "providers")
            li_elements = providers_div.find_elements(By.TAG_NAME, "li")

            points_array = []

            for li_element in li_elements:

                points_div = li_element.find_element(By.CLASS_NAME, "points")
                points_value = points_div.text
                points_array.append(points_value)

            as_points=points_array[0]
            marca_points=points_array[1]
            mundo_deportivo_points=points_array[2]
            
            #### OBTENER PARTIDO ANTERIOR ####

            # Encontrar el div principal con la clase "player-match"
            player_match_div = driver.find_element(By.CLASS_NAME, "player-match")

            # Encontrar los subelementos dentro del div principal
            team_1 = player_match_div.find_element(By.CLASS_NAME, "left").find_element(By.CLASS_NAME, "team").text
            goals_team_1 = [int(goal.text) for goal in player_match_div.find_elements(By.CLASS_NAME, "goals")][0]  
            goals_team_2 = [int(goal.text) for goal in player_match_div.find_elements(By.CLASS_NAME, "goals")][1]  
            team_2 = player_match_div.find_element(By.CLASS_NAME, "right").find_element(By.CLASS_NAME, "team").text

            if team_1 == equipo:
                ultimo_rival=team_2

                if goals_team_1 > goals_team_2:
                    result = "Win"
                elif goals_team_1 < goals_team_2:  
                    result = "Loss"
                else:
                    result = "Draw"
            else:
                ultimo_rival=team_1

                if goals_team_1 > goals_team_2:
                    result = "Loss"
                elif goals_team_1 < goals_team_2:  
                    result = "Win"
                else:
                    result = "Draw"

            driver.back()
            
        except:
            final_points=None
            as_points=None
            marca_points=None
            mundo_deportivo_points=None
            ultimo_rival=None
            result=None
            
    else:
        final_points="NA"
        as_points="NA"
        marca_points="NA"
        mundo_deportivo_points="NA"
        ultimo_rival="NA"
        result="NA"
    
    
    #### IMPRIMIR TODOS LOS DATOS ####
    print("____________________________________")
    print(index," -",nombre.text,apellido.text)
    print("Valor: ",valor)
    print("Posición: ",posicion)
    print("Equipo: ",equipo)
    
    print("- - - - - - - - - - - - - - - - - -")
   
    print("Puntuación Fantasy:", final_points)
    print("Puntuación Fantasy:", as_points)
    print("Puntuación Marca:", marca_points)
    print("Puntuación Mundo Deportivo:", mundo_deportivo_points)
 
    print("- - - - - - - - - - - - - - - - - -")
    
    print("Último rival: ",ultimo_rival)
    print("Resultado del partdio: ",result)
    print("Próximo rival: ",proximo_rival)
    print("Próximo partido es local: ",local)
    print(f"Media en casa: {media_puntos_local}")
    print(f"Media fuera: {media_puntos_visitante}")
    print(f"Edad: {edad}")
    print(f"Altura: {altura}")
    print(f"Peso: {peso}")
    
    nombre_archivo="output_scraper/"+jornada_absolute+".xlsx"
    jugador = nombre.text + " " + apellido.text


    # Crear la consulta SQL para insertar los datos
    sql_insert = """
    INSERT INTO informacion_jugadores (nombre, apellido, valor, posicion, equipo, puntuacion_fantasy, puntuacion_as, 
    puntuacion_marca, puntuacion_mundo_deportivo, ultimo_rival, resultado_partido, proximo_rival, proximo_partido_es_local, 
    media_en_casa, media_fuera, edad, altura, peso)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Ejecutar la consulta SQL
    cursor.execute(sql_insert, (nombre.text, apellido.text, valor, posicion, equipo, final_points, as_points, marca_points,
                                 mundo_deportivo_points, ultimo_rival, result, proximo_rival, local, media_puntos_local,
                                 media_puntos_visitante, edad, altura, peso))

    # Hacer commit para guardar los cambios en la base de datos
    conexion.commit()

    # Cerrar cursor y conexión
    cursor.close()
    conexion.close()
